# About

This notebook parses the player selections into a tidy format

In [1]:
import pandas as pd

In [2]:
import re

In [3]:
# Reading in the raw data
selections_raw = pd.read_csv('data/player_selections.csv',index_col=0)

In [4]:
# Compiling the regex
regex = re.compile('\s(C|C|W|UTIL|D|G|CPT|FLEX)\s')
startRegex = re.compile('^(C|C|W|UTIL|D|G|CPT|FLEX)\s')

In [5]:
numRows = len(selections_raw)

In [6]:
# Creating a list to hold the raw data
parsedsSelections = []


for index, values in selections_raw.iterrows():
    if index % 10000 == 0:
        print((index / numRows)*100)
    selections = {}
    selections['contestID'] = values['contestID']
    selections['entryID'] = values['entryID']
    selections['entryName'] = values['entryName']
    
    selectionString = values['lineup']
    
    selectionString = selections_raw.iloc[1613]['lineup']
    
#     print(index, selectionString)
#     print('**********************************')
    
    matchIndexes = [match_span.span() for match_span in regex.finditer(selectionString)]
    
    selection = []
    
    firstMatchIndexes = startRegex.match(selectionString).span()
           
    # First Player  
    selection.append({'position':selectionString[0:firstMatchIndexes[1]-1], 
                      'player':selectionString[firstMatchIndexes[1]:matchIndexes[0][0]]})
    
    for i in range(0, len(matchIndexes)):
        if i < len(matchIndexes) - 1:
            # Player 2 to n-1
            
            selection.append(
                {
                    'position':selectionString[matchIndexes[i][0]+1:matchIndexes[i][1]-1], 
                    'player':selectionString[matchIndexes[i][1]:matchIndexes[i+1][0]]
                }
            )
        else:
            # Player n
            
            selection.append(
                {
                    'position':selectionString[matchIndexes[i][0]+1:matchIndexes[i][1]-1], 
                    'player':selectionString[matchIndexes[i][1]:]
                }
            )
                
    selections['selection'] = selection
    
    parsedsSelections.append(selections)
    
            
#     break

parsedsSelectionsDF = pd.DataFrame(parsedsSelections)

0.0
0.5090708794657607
1.0181417589315214
1.527212638397282
2.0362835178630427
2.5453543973288033
3.054425276794564
3.563496156260325
4.0725670357260855
4.5816379151918465
5.090708794657607
5.599779674123368
6.108850553589128
6.617921433054888
7.12699231252065
7.63606319198641
8.145134071452171
8.654204950917931
9.163275830383693
9.672346709849453
10.181417589315213
10.690488468780973
11.199559348246735
11.708630227712495
12.217701107178256
12.726771986644017
13.235842866109776
13.744913745575538
14.2539846250413
14.763055504507058
15.27212638397282
15.781197263438582
16.290268142904342
16.799339022370102
17.308409901835862
17.817480781301622
18.326551660767386
18.835622540233143
19.344693419698906
19.853764299164666
20.362835178630426
20.871906058096187
21.380976937561947
21.890047817027707
22.39911869649347
22.908189575959227
23.41726045542499
23.92633133489075
24.43540221435651
24.94447309382227
25.453543973288035
25.962614852753795
26.47168573221955
26.980756611685315
27.4898274911

In [7]:
# Exploding the selections into individual rows
parsedsSelectionsDF = parsedsSelectionsDF.explode('selection')

In [8]:
# Expanding the selection column into 2 columns
parsedsSelectionsDF = pd.concat(
    [
        parsedsSelectionsDF.reset_index(drop=True),
        pd.json_normalize(parsedsSelectionsDF['selection'])
    ],
    axis=1)

In [9]:
# Dropping unneccessary column
playerSelections = parsedsSelectionsDF.drop('selection',axis=1)

In [10]:
playerSelections.to_csv('transformed_data/playerSelections.csv')